In [1]:
pip install python-chess pandas


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

# Load the CSV file
file_path = r"C:\Users\hiddensardar\Downloads\Structured_PGN_Games_Data__Improved_.csv"
data = pd.read_csv(file_path)

# Display the first few rows
data.head()


,Event,Date,Round,White,Black,Result,Board,TimeControl,WhiteFideId,BlackFideId,WhiteElo,BlackElo,WhiteTeam,BlackTeam,WhiteClock,BlackClock,Moves
0,January 23 Late 2024,2024.01.23,7,"Nakamura, Hikaru","Vlassov, Nikolai",1-0,26.0,180+1,2016192.0,4103360.0,3279.0,2604.0,USA,FID,0:00:22,0:00:00,1. d4 {[%clk 0:03:00]} Nf6 {[%clk 0:02:59]} 2...
1,January 23 Late 2024,2024.01.23,3,"Nakamura, Hikaru","Kushko, Dmitriy",1-0,1.0,180+1,2016192.0,14162261.0,3304.0,2856.0,USA,UKR,0:00:41,0:00:02,1. Nf3 {[%clk 0:03:00]} Nf6 {[%clk 0:02:59]} ...
2,January 23 Late 2024,2024.01.23,1,"Nakamura, Hikaru",Senthil Maran K,1-0,1.0,180+1,2016192.0,25035681.0,3304.0,2544.0,USA,IND,0:00:47,0:00:01,1. Nf3 {[%clk 0:03:00]} d5 {[%clk 0:02:57]} 2...
3,January 23 Early 2024,2024.01.23,10,"Nakamura, Hikaru","Lazavik, Denis",1-0,1.0,180+1,2016192.0,13515110.0,3306.0,3098.0,USA,BLR,0:00:36,0:00:00,1. d4 {[%clk 0:03:00]} Nf6 {[%clk 0:02:59]} 2...
4,January 23 Early 2024,2024.01.23,9,"Nakamura, Hikaru","Drygalov, Sergey",1-0,2.0,180+1,2016192.0,24171743.0,3303.0,3035.0,USA,RUS,0:00:10,0:00:00,1. d4 {[%clk 0:03:00]} d6 {[%clk 0:02:59]} 2....


In [3]:
# Select relevant columns for modeling
columns_of_interest = [
    'White', 'Black', 'Result', 'TimeControl', 'WhiteElo', 'BlackElo', 'Moves', 'WhiteClock', 'BlackClock'
]

# Filter the dataframe to include only the necessary columns
chess_data = data[columns_of_interest]

# Display the cleaned dataset
chess_data.head()


,White,Black,Result,TimeControl,WhiteElo,BlackElo,Moves,WhiteClock,BlackClock
0,"Nakamura, Hikaru","Vlassov, Nikolai",1-0,180+1,3279.0,2604.0,1. d4 {[%clk 0:03:00]} Nf6 {[%clk 0:02:59]} 2...,0:00:22,0:00:00
1,"Nakamura, Hikaru","Kushko, Dmitriy",1-0,180+1,3304.0,2856.0,1. Nf3 {[%clk 0:03:00]} Nf6 {[%clk 0:02:59]} ...,0:00:41,0:00:02
2,"Nakamura, Hikaru",Senthil Maran K,1-0,180+1,3304.0,2544.0,1. Nf3 {[%clk 0:03:00]} d5 {[%clk 0:02:57]} 2...,0:00:47,0:00:01
3,"Nakamura, Hikaru","Lazavik, Denis",1-0,180+1,3306.0,3098.0,1. d4 {[%clk 0:03:00]} Nf6 {[%clk 0:02:59]} 2...,0:00:36,0:00:00
4,"Nakamura, Hikaru","Drygalov, Sergey",1-0,180+1,3303.0,3035.0,1. d4 {[%clk 0:03:00]} d6 {[%clk 0:02:59]} 2....,0:00:10,0:00:00


In [4]:
import re

# Function to extract moves only (without clock annotations)
def extract_moves(move_str):
    # Check if the move_str is already a list (no need to process further)
    if isinstance(move_str, list):
        return move_str
    # If it's a string, extract the moves
    moves = re.findall(r'\d+\.\s*([a-zA-Z0-9+#=]+)', move_str)
    return moves

# Updated function to handle float values in clock columns
def clock_to_seconds(clock_str):
    # Convert the clock to a string first
    if isinstance(clock_str, float):
        clock_str = str(clock_str)
    
    if clock_str and clock_str != '0:00:00':
        # Handle cases where the clock might be missing or invalid
        try:
            h, m, s = map(int, clock_str.split(':'))
            return h * 3600 + m * 60 + s
        except ValueError:
            return 0
    return 0

# Apply the functions to the dataset
chess_data['Moves'] = chess_data['Moves'].apply(extract_moves)
chess_data['WhiteClockSeconds'] = chess_data['WhiteClock'].apply(clock_to_seconds)
chess_data['BlackClockSeconds'] = chess_data['BlackClock'].apply(clock_to_seconds)

# Drop the original clock columns
chess_data = chess_data.drop(columns=['WhiteClock', 'BlackClock'])

# Display the updated dataset
chess_data.head()


C:\Users\hiddensardar\AppData\Local\Temp\ipykernel_28868\607046412.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chess_data['Moves'] = chess_data['Moves'].apply(extract_moves)
C:\Users\hiddensardar\AppData\Local\Temp\ipykernel_28868\607046412.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chess_data['WhiteClockSeconds'] = chess_data['WhiteClock'].apply(clock_to_seconds)
C:\Users\hiddensardar\AppData\Local\Temp\ipykernel_28868\607046412.py:30: SettingWithCopyWarning: 
A value is trying to be set

,White,Black,Result,TimeControl,WhiteElo,BlackElo,Moves,WhiteClockSeconds,BlackClockSeconds
0,"Nakamura, Hikaru","Vlassov, Nikolai",1-0,180+1,3279.0,2604.0,"[d4, Bg5, e4, Bxf6, Nc3, Qd3, e5, a3, Qxc3, dx...",22,0
1,"Nakamura, Hikaru","Kushko, Dmitriy",1-0,180+1,3304.0,2856.0,"[Nf3, b3, c4, Qc2, Bb2, d3, Nbd2, h3, g4, Nh4,...",41,2
2,"Nakamura, Hikaru",Senthil Maran K,1-0,180+1,3304.0,2544.0,"[Nf3, c4, e3, b3, Bb2, d4, Nbd2, Bd3, e4, cxd5...",47,1
3,"Nakamura, Hikaru","Lazavik, Denis",1-0,180+1,3306.0,3098.0,"[d4, Nf3, c4, Nc3, Bf4, e3, c5, Bd3, b4, a3, B...",36,0
4,"Nakamura, Hikaru","Drygalov, Sergey",1-0,180+1,3303.0,3035.0,"[d4, e4, Nc3, f4, Qd2, dxc5, cxd6, Bd3, e5, Nc...",10,0


### work on the Elo difference (WhiteElo - BlackElo), which will help analyze the players' rating differences. We can then focus on more model-specific steps.

In [5]:
# Calculate Elo difference
chess_data['EloDifference'] = chess_data['WhiteElo'] - chess_data['BlackElo']

# Display the updated dataset
chess_data.head()


,White,Black,Result,TimeControl,WhiteElo,BlackElo,Moves,WhiteClockSeconds,BlackClockSeconds,EloDifference
0,"Nakamura, Hikaru","Vlassov, Nikolai",1-0,180+1,3279.0,2604.0,"[d4, Bg5, e4, Bxf6, Nc3, Qd3, e5, a3, Qxc3, dx...",22,0,675.0
1,"Nakamura, Hikaru","Kushko, Dmitriy",1-0,180+1,3304.0,2856.0,"[Nf3, b3, c4, Qc2, Bb2, d3, Nbd2, h3, g4, Nh4,...",41,2,448.0
2,"Nakamura, Hikaru",Senthil Maran K,1-0,180+1,3304.0,2544.0,"[Nf3, c4, e3, b3, Bb2, d4, Nbd2, Bd3, e4, cxd5...",47,1,760.0
3,"Nakamura, Hikaru","Lazavik, Denis",1-0,180+1,3306.0,3098.0,"[d4, Nf3, c4, Nc3, Bf4, e3, c5, Bd3, b4, a3, B...",36,0,208.0
4,"Nakamura, Hikaru","Drygalov, Sergey",1-0,180+1,3303.0,3035.0,"[d4, e4, Nc3, f4, Qd2, dxc5, cxd6, Bd3, e5, Nc...",10,0,268.0


In [8]:
# other

In [6]:
# Selecting relevant features for the model
model_data = chess_data[['Moves', 'EloDifference', 'TimeControl', 'WhiteClockSeconds', 'BlackClockSeconds']].copy()

# Display the model dataset
model_data.head()


,Moves,EloDifference,TimeControl,WhiteClockSeconds,BlackClockSeconds
0,"[d4, Bg5, e4, Bxf6, Nc3, Qd3, e5, a3, Qxc3, dx...",675.0,180+1,22,0
1,"[Nf3, b3, c4, Qc2, Bb2, d3, Nbd2, h3, g4, Nh4,...",448.0,180+1,41,2
2,"[Nf3, c4, e3, b3, Bb2, d4, Nbd2, Bd3, e4, cxd5...",760.0,180+1,47,1
3,"[d4, Nf3, c4, Nc3, Bf4, e3, c5, Bd3, b4, a3, B...",208.0,180+1,36,0
4,"[d4, e4, Nc3, f4, Qd2, dxc5, cxd6, Bd3, e5, Nc...",268.0,180+1,10,0


In [7]:
from sklearn.model_selection import train_test_split

# Splitting the data into training and testing sets
X = model_data.drop(columns=['Moves'])  # Features
y = model_data['Moves']  # Target variable (you might want to change this based on your actual target)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shapes of the train and test sets
(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


((1387, 4), (347, 4), (1387,), (347,))

In [11]:
# Next Steps 
"""
Vectorizing Moves: Since the Moves are currently in list format, we'll need to convert them into a format suitable for machine learning. A common approach is to use Count Vectorization or TF-IDF Vectorization.

Training the Model: After vectorizing the moves, we can select a model to train on the dataset.

Step 1: Vectorizing Moves
 """

"\nVectorizing Moves: Since the Moves are currently in list format, we'll need to convert them into a format suitable for machine learning. A common approach is to use Count Vectorization or TF-IDF Vectorization.\n\nTraining the Model: After vectorizing the moves, we can select a model to train on the dataset.\n\nStep 1: Vectorizing Moves\n "

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

# Flatten the list of moves for vectorization
model_data['Moves'] = model_data['Moves'].apply(lambda x: ' '.join(x))  # Join moves into a single string

# Create the CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the training moves
X_train_moves = vectorizer.fit_transform(model_data.loc[X_train.index, 'Moves'])

# Transform the test moves
X_test_moves = vectorizer.transform(model_data.loc[X_test.index, 'Moves'])

# Convert to DataFrame for easier concatenation
X_train_moves_df = pd.DataFrame(X_train_moves.toarray(), columns=vectorizer.get_feature_names_out(), index=X_train.index)
X_test_moves_df = pd.DataFrame(X_test_moves.toarray(), columns=vectorizer.get_feature_names_out(), index=X_test.index)

# Combine the original features with the vectorized moves
X_train_combined = pd.concat([X_train.reset_index(drop=True), X_train_moves_df.reset_index(drop=True)], axis=1)
X_test_combined = pd.concat([X_test.reset_index(drop=True), X_test_moves_df.reset_index(drop=True)], axis=1)

# Display the combined dataset shape
X_train_combined.shape, X_test_combined.shape


((1387, 1242), (347, 1242))

In [13]:
# Step 2: Training the Random Forest Classifier

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Prepare the target variable
y_train = model_data.loc[X_train.index, 'Result']
y_test = model_data.loc[X_test.index, 'Result']

# Initialize the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the model on the training data
rf_classifier.fit(X_train_combined, y_train)

# Predict on the test set
y_pred = rf_classifier.predict(X_test_combined)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Display the accuracy and classification report
accuracy, report



KeyError: 'Result'

In [18]:
# Error resolving

In [15]:
# Display the columns of model_data
model_data.columns

Index(['Moves', 'EloDifference', 'TimeControl', 'WhiteClockSeconds',
       'BlackClockSeconds'],
      dtype='object')

In [16]:
# Assuming chess_data is your original DataFrame
model_data = chess_data[['Moves', 'EloDifference', 'TimeControl', 'WhiteClockSeconds', 'BlackClockSeconds', 'Result']].copy()

In [35]:
# Display unique values in the 'Result' column
unique_results = chess_data['Result'].unique()
print(unique_results)

['1-0' '0-1' '1/2-1/2' nan]


In [40]:

chess_data["Result"].nunique()

3

In [42]:
## tryy 1

In [41]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

# Step 1: Convert 'Result' column to string and replace
model_data['Result'] = model_data['Result'].astype(str).replace({'1-0': 1, '0-1': 0, '1/2-1/2': 0.5})

# Step 2: Create a new column for the count of moves
model_data['MoveCount'] = model_data['Moves'].apply(len)

# Step 3: Function to convert TimeControl to numeric features
def parse_time_control(time_control):
    if '+' in time_control:
        base_time, increment = time_control.split('+')
        return int(base_time), int(increment)
    else:
        return int(time_control), 0  # No increment

# Step 4: Create new columns for base time and increment
model_data[['BaseTime', 'Increment']] = model_data['TimeControl'].apply(parse_time_control).apply(pd.Series)

# Step 5: Features and target variable
X = model_data[['MoveCount', 'EloDifference', 'BaseTime', 'Increment', 'WhiteClockSeconds', 'BlackClockSeconds']]
y = model_data['Result']

# Step 6: Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 7: Initialize the model
model = RandomForestClassifier()

# Step 8: Fit the model
model.fit(X_train, y_train)

# Step 9: Make predictions
predictions = model.predict(X_test)

# Step 10: Evaluate the model
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)


ValueError: invalid literal for int() with base 10: '40/7200:1800'

In [43]:
# TRY 2

In [44]:
print(model_data['TimeControl'].unique())


['180+1' '600+2' '180+2' '300+1' '60+1' '1500+10' '600' '40/7200:1800+30'
 '900+10' '40/7200:0+10' '60' '40/6000+30:20/3000+30' '40/600:0+1'
 '40/5400+30:1800+30' '5400+30' '900+3' '15+3' '900+2' '600+5' '300+3'
 '180']


In [47]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

# Assume model_data is already defined and looks like your DataFrame

# Step 1: Convert 'Result' column to string and replace
model_data['Result'] = model_data['Result'].astype(str).replace({'1-0': 1, '0-1': 0, '1/2-1/2': 0.5})

# Step 2: Create a new column for the count of moves
model_data['MoveCount'] = model_data['Moves'].apply(len)

# Step 3: Update the function to convert TimeControl to numeric features
def parse_time_control(time_control):
    if isinstance(time_control, str):
        # Handle formats with both '+' and ':'
        if '+' in time_control:
            parts = time_control.split('+')
            base_time = parts[0]  # Take the first part as base time
            increment = parts[1] if len(parts) > 1 else '0'  # Use the second part if it exists
        else:
            base_time, increment = time_control, '0'  # Fallback for single numbers or formats without '+'

        # Further handle cases where base_time has a '/' or ':'
        if '/' in base_time:
            base_time = base_time.split('/')[0]  # Use the first part of the fraction
        if ':' in base_time:
            base_time = base_time.split(':')[0]  # Use the first part before ':'

        # Now handle the increment part, which may also contain ':'
        if ':' in increment:
            increment = increment.split(':')[0]  # Use the first part before ':'
        
        try:
            return int(base_time), int(increment)  # Convert both to int and return
        except ValueError:
            return 0, 0  # If conversion fails, return defaults
        
    return 0, 0  # If it's NaN or not a string

# Step 4: Create new columns for base time and increment
model_data[['BaseTime', 'Increment']] = model_data['TimeControl'].apply(parse_time_control).apply(pd.Series)

# Step 5: Features and target variable
X = model_data[['MoveCount', 'EloDifference', 'BaseTime', 'Increment', 'WhiteClockSeconds', 'BlackClockSeconds']]
y = model_data['Result']

# Step 6: Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 7: Initialize the model
model = RandomForestClassifier()

# Step 8: Fit the model
model.fit(X_train, y_train)

# Step 9: Make predictions
predictions = model.predict(X_test)

# Step 10: Evaluate the model
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)


Accuracy: 0.7703488372093024
Classification Report:
               precision    recall  f1-score   support

         0.0       0.54      0.30      0.38        44
         0.5       0.59      0.50      0.54        72
         1.0       0.83      0.95      0.89       228

    accuracy                           0.77       344
   macro avg       0.66      0.58      0.60       344
weighted avg       0.75      0.77      0.75       344



In [48]:
"""lassification Report Analysis
Precision: This indicates the proportion of positive identifications that were actually correct.

For class 0.0: 54% of the predictions were correct.
For class 0.5: 59% of the predictions were correct.
For class 1.0: 83% of the predictions were correct.
Recall: This measures the proportion of actual positives that were identified correctly.

For class 0.0: The model identified 30% of actual 0.0 correctly.
For class 0.5: The model identified 50% of actual 0.5 correctly.
For class 1.0: The model identified 95% of actual 1.0 correctly.
F1-Score: This is the harmonic mean of precision and recall, providing a balance between the two.

For class 0.0: 0.38
For class 0.5: 0.54
For class 1.0: 0.89
Insights:
Class Imbalance: There seems to be a significant imbalance in your classes, particularly for class 1.0, which the model predicts well. Classes 0.0 and 0.5 have lower precision and recall, which may indicate that the model is struggling to identify those classes correctly.
Weighted Average: The weighted average metrics indicate a decent performance overall, but improvements could still be made."""

'lassification Report Analysis\nPrecision: This indicates the proportion of positive identifications that were actually correct.\n\nFor class 0.0: 54% of the predictions were correct.\nFor class 0.5: 59% of the predictions were correct.\nFor class 1.0: 83% of the predictions were correct.\nRecall: This measures the proportion of actual positives that were identified correctly.\n\nFor class 0.0: The model identified 30% of actual 0.0 correctly.\nFor class 0.5: The model identified 50% of actual 0.5 correctly.\nFor class 1.0: The model identified 95% of actual 1.0 correctly.\nF1-Score: This is the harmonic mean of precision and recall, providing a balance between the two.\n\nFor class 0.0: 0.38\nFor class 0.5: 0.54\nFor class 1.0: 0.89\nInsights:\nClass Imbalance: There seems to be a significant imbalance in your classes, particularly for class 1.0, which the model predicts well. Classes 0.0 and 0.5 have lower precision and recall, which may indicate that the model is struggling to ident